<a href="https://colab.research.google.com/github/SteveChengChen/data-analysis/blob/main/EECS_6895_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `ChengChen` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

In [5]:
from transformers import AutoTokenizer
import transformers
import torch

chat_model_id = "meta-llama/Llama-2-7b-chat-hf"

In [6]:
chat_tokenizer = AutoTokenizer.from_pretrained(
    chat_model_id,
    use_auth_token=True
)

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
from transformers import AutoModelForCausalLM

chat_model = AutoModelForCausalLM.from_pretrained(
    chat_model_id,
    torch_dtype=torch.float16,
    device_map="auto",
    use_auth_token=True
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [9]:
from transformers import pipeline

chat_pipeline = pipeline(
    "text-generation",
    model=chat_model,
    tokenizer=chat_tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    temperature=0.7,
    top_k=50,
    do_sample=True,
    pad_token_id=chat_tokenizer.eos_token_id
)

Device set to use cuda:0


In [10]:
conversation_history = ""
last_answer = ""

system_prompt = """You are an assistant.
Always answer helpfully.
If you don't know the answer, say you don't know.
"""

In [11]:
def chat(user_query):
    """
    Sends `user_query` to Llama 2 Chat model,
    retaining conversation history across calls.
    """
    global conversation_history, last_answer

    if conversation_history == "":

        conversation_history = (
            f"<s>[INST] <<SYS>>\n{system_prompt}\n<</SYS>>\n\n{user_query} [/INST]"
        )
    else:

        conversation_history += (
            f" {last_answer}</s><s>[INST] {user_query} [/INST]"
        )

    out = chat_pipeline(
        conversation_history,
        max_new_tokens=256,
        truncation=True
    )
    full_output = out[0]["generated_text"]

    new_answer = full_output[len(conversation_history):].strip()

    last_answer = new_answer
    print("Assistant:", new_answer)
    return new_answer

In [12]:
user_input_1 = "Hello, how are you today?"
chat(user_input_1)

user_input_2 = "Can you summarize the current large language models?"
chat(user_input_2)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Assistant: Hello! I'm just an AI, I don't have feelings or emotions like humans do, so I don't have a physical body or experience emotions like happiness or sadness. However, I'm here to help you with any questions or tasks you may have, so feel free to ask me anything! Is there something specific you need help with today?
Assistant: Certainly! Large language models are a class of artificial intelligence (AI) models that are trained on vast amounts of text data to generate language outputs that are coherent and natural-sounding. Here's a summary of some of the current large language models:

1. BERT (Bidirectional Encoder Representations from Transformers): BERT is a popular language model developed by Google in 2018. It uses a multi-layer bidirectional transformer encoder to generate contextualized representations of words in a sentence. BERT has been fine-tuned for various natural language processing (NLP) tasks such as sentiment analysis, question-answering, and text classification.

"Certainly! Large language models are a class of artificial intelligence (AI) models that are trained on vast amounts of text data to generate language outputs that are coherent and natural-sounding. Here's a summary of some of the current large language models:\n\n1. BERT (Bidirectional Encoder Representations from Transformers): BERT is a popular language model developed by Google in 2018. It uses a multi-layer bidirectional transformer encoder to generate contextualized representations of words in a sentence. BERT has been fine-tuned for various natural language processing (NLP) tasks such as sentiment analysis, question-answering, and text classification.\n2. RoBERTa (Robustly Optimized BERT Pretraining Approach): RoBERTa is a variant of BERT that was developed in 2020 by Facebook AI. It uses a modified version of the BERT architecture and training procedure to improve its performance on downstream NLP tasks. RoBERTa has achieved state-of-the-art results on several benchmark datase

In [1]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `ChengChen` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

In [2]:
!pip install bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstal

In [3]:
!pip install faiss-gpu-cu12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 MB 48.5 MB/s eta 0:00:00


In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.3 MB/s eta 0:00:00


In [33]:
english_text = """\
Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., ChatGPT and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once something becomes useful enough and common enough it's not labeled AI anymore."

Various subfields of AI research are centered around particular goals and the use of particular tools. The traditional goals of AI research include reasoning, knowledge representation, planning, learning, natural language processing, perception, and support for robotics.[a] General intelligence—the ability to complete any task performed by a human on an at least equal level—is among the field's long-term goals. To reach these goals, AI researchers have adapted and integrated a wide range of techniques, including search and mathematical optimization, formal logic, artificial neural networks, and methods based on statistics, operations research, and economics.[b] AI also draws upon psychology, linguistics, philosophy, neuroscience, and other fields.

Artificial intelligence was founded as an academic discipline in 1956, and the field went through multiple cycles of optimism throughout its history, followed by periods of disappointment and loss of funding, known as AI winters. Funding and interest vastly increased after 2012 when deep learning outperformed previous AI techniques. This growth accelerated further after 2017 with the transformer architecture, and by the early 2020s many billions of dollars were being invested in AI and the field experienced rapid ongoing progress in what has become known as the AI boom. The emergence of advanced generative AI in the midst of the AI boom and its ability to create and modify content exposed several unintended consequences and harms in the present and raised concerns about the risks of AI and its long-term effects in the future, prompting discussions about regulatory policies to ensure the safety and benefits of the technology.


"""

chinese_text = """\
人工智能，称机器智能，指由人制造出来的机器所表现出来的智能。通常人工智能是指用普通计算机程序来呈现人类智能的技术。该词也指出研究这样的智能系统是否能够实现，以及如何实现。同时，随着医学、神经科学、机器人学及统计学等方面的发展，普遍认为人类的部分职业也逐渐被其取代。

人工智能于一般教材中的定义领域是“智能主体的研究与设计”，智能主体指一个可以观察周遭环境并作出行动以达致目标的系统。约翰·麦卡锡于1955年的定义是“制造智能机器的科学与工程”。安德烈亚斯·卡普兰和迈克尔·海恩莱因将人工智能定义为“系统正确解释外部数据，从这些数据中学习，并利用这些知识通过灵活适应实现特定目标和任务的能力”。人工智能可以定义为模仿人类与人类思维相关的认知功能的机器或计算机，如学习和解决问题。人工智能是计算机科学的一个分支，它感知其环境并采取行动，最大限度地提高其成功机会。此外，人工智能能够从过去的经验中学习，做出合理的决策，并快速回应。因此，人工智能研究人员的科学目标是通过构建具有象征意义的推理或推理的计算机程序来理解智能。人工智能的四个主要组成部分是：

专家系统：作为专家处理正在审查的情况，并产生预期或预期的绩效。
启发式算法：包括评估小范围的解决方案，并可能涉及一些猜测，以找到接近最佳的解决方案。
自然语言处理：在自然语言中实现人机之间的交流。
计算机视觉：自动生成识别形状和功能的能力。
人工智能的研究是高度技术性和专业的，各分支领域都是深入且各不相通的，因而涉及范围极广。人工智能的研究可以分为几个技术问题。其分支领域主要集中在解决具体问题，其中之一是，如何使用各种不同的工具完成特定的应用程序。

AI的核心问题包括建构能够跟人类似甚至超卓的推理、知识、计划、学习、交流、感知、移动 、移物、使用工具和操控机械的能力等。通用人工智能目前仍然是该领域的长远目标。目前弱人工智能已经有初步成果，甚至在一些影像识别、语言分析、棋类游戏等等单方面的能力达到了超越人类的水平，而且人工智能的通用性代表着，能解决上述的问题的是一样的AI程序，无须重新开发算法就可以直接使用现有的AI完成任务，与人类的处理能力相同，但达到具备思考能力的统合强人工智能还需要时间研究，比较流行的方法包括统计方法，计算智能和传统意义的AI。目前有大量的工具应用了人工智能，其中包括搜索和数学优化、逻辑推演。而基于仿生学、认知心理学，以及基于概率论和经济学的算法等等也在逐步探索当中
"""

with open("english_text.txt", "w", encoding="utf-8") as f_en:
    f_en.write(english_text)

with open("chinese_text.txt", "w", encoding="utf-8") as f_cn:
    f_cn.write(chinese_text)

Successfully created english_text.txt and chinese_text.txt!


In [34]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [35]:
quant_config = BitsAndBytesConfig(load_in_4bit=True)

model_name = "meta-llama/Llama-3.2-1B"

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)


In [47]:
loader_en = TextLoader("english_text.txt")
loader_cn = TextLoader("chinese_text.txt")

docs_en = loader_en.load()
docs_cn = loader_cn.load()

print(f"Loaded {len(docs_en)} English doc(s) and {len(docs_cn)} Chinese doc(s).")

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs_en = splitter.split_documents(docs_en)
split_docs_cn = splitter.split_documents(docs_cn)

print(f"English docs split into {len(split_docs_en)} chunks.")
print(f"Chinese docs split into {len(split_docs_cn)} chunks.")

Loaded 1 English doc(s) and 1 Chinese doc(s).
English docs split into 7 chunks.
Chinese docs split into 3 chunks.


In [48]:
embed_model = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/distiluse-base-multilingual-cased-v2"
)

vector_store_en = FAISS.from_documents(split_docs_en, embed_model)
vector_store_cn = FAISS.from_documents(split_docs_cn, embed_model)

retriever_en = vector_store_en.as_retriever(search_kwargs={"k": 2})
retriever_cn = vector_store_cn.as_retriever(search_kwargs={"k": 2})

In [38]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

quant_config = BitsAndBytesConfig(load_in_4bit=True)
model_name = "meta-llama/Llama-3.2-1B"

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

In [49]:
def is_chinese_query(query: str) -> bool:

    for ch in query:
        if "\u4e00" <= ch <= "\u9fff":
            return True
    return False

In [50]:
def rag_qa_chatbot(query, retriever, model, tokenizer):

    if is_chinese_query(query):
        retriever = retriever_cn
    else:
        retriever = retriever_en

    docs = retriever.get_relevant_documents(query)

    context_text = "\n".join([doc.page_content for doc in docs])

    prompt = f"""You are a multilingual assistant.
Use the following context to answer the user's question.
If you don't see the answer in the context, just say "I don't know."

CONTEXT:
{context_text}

QUESTION: {query}
ANSWER:
"""

    print("\n PROMPT (DEBUG) ")
    print(prompt)
    print(" END PROMPT \n")

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        output_ids = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7
        )

    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    answer = output_text[len(prompt):].strip()

    print("CONTEXT (retrieved):")
    print(context_text)
    print("\nQUESTION:", query)
    print("ANSWER:", answer, "\n")

    return answer

In [51]:
def interactive_rag_chat(retriever, model, tokenizer):
    print("\nWelcome to the Multilingual RAG Chatbot!")
    print("Type 'exit' to quit.\n")

    while True:
        user_query = input("User: ")
        if user_query.strip().lower() == "exit":
            print("Goodbye!")
            break

        answer = rag_qa_chatbot(user_query, retriever, model, tokenizer)
        print("Assistant:", answer)
        print("-" * 50)

In [52]:
print("\n DEMO: English Query \n")
demo_query_en = "What is artificial intelligence?"
answer_en = rag_qa_chatbot(demo_query_en, retriever, model_4bit, tokenizer)
print(f"Final Answer (English): {answer_en}")

print("\n DEMO: Chinese Query \n")
demo_query_cn = "什么是人工智能？"
answer_cn = rag_qa_chatbot(demo_query_cn, retriever, model_4bit, tokenizer)
print(f"Final Answer (Chinese): {answer_cn}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



 DEMO: English Query 


 PROMPT (DEBUG) 
You are a multilingual assistant.
Use the following context to answer the user's question.
If you don't see the answer in the context, just say "I don't know."

CONTEXT:
Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
Artificial intelligence was founded as an academic discipline in 1956, and the field went through multiple cycles of optimism throughout its history, followed by periods of disappointment and loss of funding, known as AI winters. Funding and interest vastly increased after 2012 when deep learning outperformed previous AI techniques. This growth accelerated further after 2017 with

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


CONTEXT (retrieved):
Artificial intelligence (AI), in its broadest sense, is intelligence exhibited by machines, particularly computer systems. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals. Such machines may be called AIs.
Artificial intelligence was founded as an academic discipline in 1956, and the field went through multiple cycles of optimism throughout its history, followed by periods of disappointment and loss of funding, known as AI winters. Funding and interest vastly increased after 2012 when deep learning outperformed previous AI techniques. This growth accelerated further after 2017 with the transformer architecture, and by the early 2020s many billions of dollars were being invested in AI and the

QUESTION: What is artificial intelligence?
ANSWER: Artificial intelligence (A